# Example classification analysis using ShinyLearner

*By Erica Suh and Stephen Piccolo*

This notebook illustrates how to perform a benchmark comparison of classification algorithms across multiple datasets. We assume the reader has a moderate level of understanding of shell scripting and Python scripting. We also assume that the user's operating system is UNIX-based.

### Preparing the data

The Penn Machine Learning Benchmarks (PMLB) repository contains a large number of datasets that can be used to test machine-learning algorithms. We can access this repository using the Python module named `pmlb`, which can be installed via `pip`.

In [ ]:
%%bash

# This step may or may not be necessary:
pip3 install --upgrade pip

# You only need to install this module once
pip3 install pmlb

For demonstration purposes, we will fetch 10 biology-related datasets from PMLB. First, define a list that indicates the unique identifier for each of these datasets.

In [6]:
datasets = ['analcatdata_aids',
            'ann-thyroid',
            'breast-cancer',
            'dermatology',
            'diabetes',
            'hepatitis',
            'iris',
            'liver-disorder',
            'molecular-biology_promoters',
            'yeast']

PMLB's [GitHub repository](https://github.com/EpistasisLab/penn-ml-benchmarks) demonstrates how `pmlb` is used in Python scripts.

ShinyLearner requires that input data files have exactly one feature named 'Class', which includes the class labels. So we must modify the PMLB data to meet this requirement. After modifying the data, we save each DataFrame to a a file with a [supported extension](https://github.com/srp33/ShinyLearner/blob/master/InputFormats.md).

In [7]:
from pmlb import fetch_data
import os
import shutil

directory = "Datasets"
if os.path.exists(directory):
    shutil.rmtree(directory)
os.makedirs(directory)

for data in datasets:
    curr_data = fetch_data(data)
    curr_data = curr_data.rename(columns={'target': 'Class'})  # Rename 'target' to 'Class'
    curr_data.to_csv('{}/{}.tsv'.format(directory, data), sep='\t', index=True)  # Save to a .tsv file

### Performing a benchmark comparison of 10 classification algorithms

For this initial analysis, we will apply 10 different classification algorithms to the 10 datasets we have prepared. We will use Monte Carlo cross validation (with *no* hyperparameter optimization). To keep the execution time reasonable, we will only do 5 iterations of Monte Carlo cross validation.

ShinyLearner is executed within a Docker container. The ShinyLearner [web application](http://bioapps.byu.edu/shinylearner/) enables us to more easily build commands for executing ShinyLearner within Docker at the command line. We used this tool to create a template command. Below we modify that template and execute ShinyLearner for each dataset. We also indicate that we want to one-hot encode (`--ohe`) and scale the data (`--scale`) and that we want to impute any missing values (`--impute`).

(*This process takes several minutes to execute. You won't see any output until the analysis has completed. To facilitate this long-running execution, you could [run this notebook at the command line](https://stackoverflow.com/a/40311709).*)

In [ ]:
%%bash

function runShinyLearner {
  dataset_file_path="$1"  
  dataset_file_name="$(basename $dataset_file_path)"
  dataset_name="${dataset_file_name/\.tsv/}"
  dataset_results_dir_path="$(pwd)/Results_Basic/$dataset_name"
  
  mkdir -p "$dataset_results_dir_path"

  docker run --rm \
    -v "$(pwd)/Datasets":/InputData \
    -v "$dataset_results_dir_path":/OutputData \
    --user $(id -u):$(id -g) \
    srp33/shinylearner:version508 \
    /UserScripts/classification_montecarlo \
      --data "$dataset_file_name" \
      --description "$dataset_name" \
      --iterations 5 \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/svm/default*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/mlr/earth/default*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/mlr/h2o.randomForest/default*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/decision_tree/default*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/logistic_regression/default*" \
      --classif-algo "/AlgorithmScripts/Classification/arff/weka/HoeffdingTree/default*" \
      --classif-algo "/AlgorithmScripts/Classification/arff/weka/MultilayerPerceptron/default*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/mlr/mlp/default*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/multilayer_perceptron/default*" \
      --classif-algo "/AlgorithmScripts/Classification/arff/weka/SimpleLogistic/default*" \
      --output-dir "/OutputData" \
      --ohe true \
      --scale true \
      --impute true \
      --verbose false
}

rm -rf Results_Basic

for dataset_file_path in ./Datasets/*.tsv
do
  runShinyLearner "$dataset_file_path"
done

### Repeating the benchmark comparison with hyperparameter optimization

ShinyLearner provides an option to optimize a classification algorithm's hyperparameters. To accomplish this, it uses nested cross validation. This process requires more computational time, but it often increases classification accuracy. In the code below, we execute the same 10 classification algorithms on the same 10 datasets. There are some differences in the code below compared to the code above:

1. We store the output in `Results_ParamsOptimized` rather than `Results_Basic`.
2. We use the `nestedclassification_montecarlo` user script rather than `classification_montecarlo`.
3. The path specified for each classification algorithm ends with `*` rather than `default*`. This tells ShinyLearner to evaluate all hyperparameter combinations, not just default ones.
4. We indicate that we want to use 5 "outer" iterations and 3 "inner" iterations (to optimize hyperparameters).

(*This process takes several hours to execute. You won't see any output until the analysis has completed. To facilitate this long-running execution, you could [run this notebook at the command line](https://stackoverflow.com/a/40311709).*)

In [ ]:
%%bash

function runShinyLearner {
  dataset_file_path="$1"
  dataset_file_name="$(basename $dataset_file_path)"
  dataset_name="${dataset_file_name/\.tsv/}"
  dataset_results_dir_path="$(pwd)/Results_ParamsOptimized/$dataset_name"
  
  mkdir -p $dataset_results_dir_path

  docker run --rm \
    -v "$(pwd)/Datasets":/InputData \
    -v "$dataset_results_dir_path":/OutputData \
    --user $(id -u):$(id -g) \
    srp33/shinylearner:version508 \
    /UserScripts/nestedclassification_montecarlo \
      --data "$dataset_file_name" \
      --description "$dataset_name" \
      --outer-iterations 5 \
      --inner-iterations 3 \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/svm/*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/mlr/earth/*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/mlr/h2o.randomForest/*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/decision_tree/*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/logistic_regression/*" \
      --classif-algo "/AlgorithmScripts/Classification/arff/weka/HoeffdingTree/*" \
      --classif-algo "/AlgorithmScripts/Classification/arff/weka/MultilayerPerceptron/*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/mlr/mlp/*" \
      --classif-algo "/AlgorithmScripts/Classification/tsv/sklearn/multilayer_perceptron/*" \
      --classif-algo "/AlgorithmScripts/Classification/arff/weka/SimpleLogistic/*" \
      --ohe true \
      --scale true \
      --impute true \
      --verbose false
}

rm -rf Results_ParamsOptimized

for dataset_file_path in ./Datasets/*.tsv
do
  runShinyLearner "$dataset_file_path"
done

### Compress output files and clean up

In [9]:
%%bash

gzip Results_Basic/*/*.tsv
gzip Results_ParamsOptimized/*/*.tsv

rm -rf Datasets

### Analyzing and visualizing the results

Please see the document called `Analyze_Results.Rmd`, which contains R code for analyzing and visualizing the results.